# The local unitary cluster Jastrow (LUCJ) ansatz

In this tutorial, we show how to use ffsim to simulate the local unitary cluster Jastrow (LUCJ) ansatz. We'll use it to calculate the ground state energy of a hydrogen molecule at a stretched bond length.

In [ ]:
import pyscf
import ffsim

# Build an H2 molecule
mol = pyscf.gto.Mole()
mol.build(
    atom=[["H", (0, 0, 0)], ["H", (0, 0, 1.8)]],
    basis="sto-6g",
)
hartree_fock = pyscf.scf.RHF(mol)
hartree_fock.kernel()

# Get molecular data and molecular Hamiltonian (one- and two-body tensors)
mol_data = ffsim.MolecularData.from_hartree_fock(hartree_fock)
norb = mol_data.norb
nelec = mol_data.nelec
mol_hamiltonian = mol_data.hamiltonian


## The unitary cluster Jastrow (UCJ) ansatz

Before describing the LUCJ, we first introduce the general unitary cluster ansatz (UCJ), which has the form

$$
  \lvert \Psi \rangle = \prod_{k = 1}^L \mathcal{W_k} e^{i \mathcal{J}_k} \mathcal{W_k^\dagger} \lvert \Phi_0 \rangle
$$

where $\lvert \Phi_0 \rangle$ is a reference state, often taken as the Hartree-Fock state, each $\mathcal{W_k}$ is an [orbital rotation](./02-orbital-rotation.ipynb), and each $\mathcal{J}_k$ is a diagonal Coulomb operator of the form

$$
    \mathcal{J} = \frac12\sum_{ij,\sigma \tau} \mathbf{J}^{\sigma \tau}_{ij} n_{i,\sigma} n_{j,\tau}.
$$

Note that this expression for the diagonal Coulomb operator is more general than the one introduced in [the previous tutorial](./03-double-factorized.ipynb) because the matrices $\mathbf{J}^{\sigma \tau}$ are indexed by the spins $\sigma$ and $\tau$. In order that the operator commutes with the total spin operator, we enforce that $\mathbf{J}^{\alpha\alpha} = \mathbf{J}^{\beta\beta}$ and $\mathbf{J}^{\alpha\beta} = \mathbf{J}^{\beta\alpha}$. As a result, we have two sets of matrices for describing the diagonal Coulomb operators: "alpha-alpha" matrices containing coefficients for terms involving the same spin, and "alpha-beta" matrices containing coefficients for terms involving different spins.

In ffsim, the UCJ ansatz operator $\prod_{k = 1}^L \mathcal{W_k} e^{i \mathcal{J}_k} \mathcal{W_k^\dagger}$ is represented by the `UCJOperator` class, which is just a dataclass that stores the diagonal Coulomb matrices and orbital rotations. A constructor method is provided to initialize the operator from a truncated double factorization of t2 amplitudes (e.g. from CCSD or MP2).

In the code cell below, we run CCSD to get the t2 amplitudes for initializing the ansatz. We'll create an ansatz operator with 2 repetitions ($L = 2$). For our reference state, we'll use the Hartree-Fock state. We use the function `apply_ucj_operator` to apply the ansatz operator to the reference state to obtain the ansatz state. Finally, we compute the energy of the ansatz state.

In [ ]:
import numpy as np
from pyscf import cc

# Get CCSD t2 amplitudes for initializing the ansatz
ccsd = cc.CCSD(hartree_fock)
_, _, t2 = ccsd.kernel()

# Construct UCJ operator
n_reps = 2
operator = ffsim.UCJOperator.from_t_amplitudes(t2, n_reps=n_reps)

# Construct the Hartree-Fock state to use as the reference state
n_alpha, n_beta = nelec
reference_state = ffsim.slater_determinant(
    norb=norb, occupied_orbitals=(range(n_alpha), range(n_beta))
)

# Apply the operator to the reference state
ansatz_state = ffsim.apply_ucj_operator(
    reference_state, operator, norb=norb, nelec=nelec
)

# Compute the energy ⟨ψ|H|ψ⟩ of the ansatz state
hamiltonian = ffsim.linear_operator(mol_hamiltonian, norb=norb, nelec=nelec)
energy = np.real(np.vdot(ansatz_state, hamiltonian @ ansatz_state))
print(f"Energy at initialialization: {energy}")

To facilitate variational optimization of the ansatz, `UCJOperator` implements methods for conversion to and from a vector of real-valued parameters. The precise relation between a parameter vector and the matrices of the UCJ operator is somewhat complicated. In short, the parameter vector stores the entries of the UCJ matrices in a non-redundant way (for the orbital rotations, the parameter vector actually stores the entries of their generators.)

The following code cell shows how one can define an objective function that takes as input a parameter vector and outputs the energy of the associated ansatz state, and then optimize this objective function using `scipy.optimize.minimize`.

In [ ]:
import scipy.optimize


def fun(x):
    # Initialize the ansatz operator from the parameter vector
    operator = ffsim.UCJOperator.from_parameters(x, norb=norb, n_reps=n_reps)
    # Apply the ansatz operator to the reference state
    final_state = ffsim.apply_ucj_operator(
        reference_state, operator, norb=norb, nelec=(n_alpha, n_beta)
    )
    # Return the energy ⟨ψ|H|ψ⟩ of the ansatz state
    return np.real(np.vdot(final_state, hamiltonian @ final_state))


result = scipy.optimize.minimize(fun, x0=operator.to_parameters())

print(f"Number of parameters: {len(result.x)}")
print(result)

## The local unitary cluster Jastrow (LUCJ) ansatz

Implementing the $e^{i \mathcal{J}_k}$ term of the UCJ ansatz requires either all-to-all connectivity or the use of a fermionic swap network, making it challenging for noisy pre-fault-tolerant quantum processors that have limited connectivity. The idea of the *local* UCJ ansatz is to impose sparsity constraints on the $\mathbf{J}^{\alpha\alpha}$ and $\mathbf{J}^{\alpha\beta}$ matrices which allow them to be implemented in constant depth on qubit topologies with limited connectivity. The constraints are specified by a list of indices indicating which matrix entries in the upper triangle are allowed to be nonzero (since the matrices are symmetric, only the upper triangle needs to be specified).

As an example, consider a square lattice qubit topology. We can place the $\alpha$ and $\beta$ orbitals in parallel lines on the lattice, with connections between these lines forming "rungs" of a ladder shape. With this setup, orbitals with the same spin are connected with a line topology, while orbitals with different spins are connected when they share the same spatial orbital. This yields the following index constraints on the $\mathbf{J}$ matrices:

$$
\begin{align*}
\mathbf{J}^{\alpha\alpha} &: \set{(p, p+1) \; , \; p = 0, \ldots, N-2} \\
\mathbf{J}^{\alpha\beta} &: \set{(p, p) \;, \; p = 0, \ldots, N-1}
\end{align*}
$$

In other words, if the $\mathbf{J}$ matrices are nonzero only at the specified indices in the upper triangle, then the $e^{i \mathcal{J}_k}$ term can be implemented on a square topology without using any swap gates, in constant depth. Of course, imposing such constraints on the ansatz makes it less expressive, so more ansatz repetitions may be required.

In the following code cell, we demonstrate the optimization of the ansatz with these constraints imposed. We still choose to use 2 repetitions, so notice that the number of parameters in the optimization has decreased from 20 to 14. In this simple example, we are still able to recover the ground state energy exactly.

In [ ]:
alpha_alpha_indices = [(p, p + 1) for p in range(norb - 1)]
alpha_beta_indices = [(p, p) for p in range(norb)]


def fun(x):
    # Initialize the ansatz operator from the parameter vector
    operator = ffsim.UCJOperator.from_parameters(
        x,
        norb=norb,
        n_reps=n_reps,
        alpha_alpha_indices=alpha_alpha_indices,
        alpha_beta_indices=alpha_beta_indices,
    )
    # Apply the ansatz operator to the reference state
    final_state = ffsim.apply_ucj_operator(
        reference_state, operator, norb=norb, nelec=(n_alpha, n_beta)
    )
    # Return the energy ⟨ψ|H|ψ⟩ of the ansatz state
    return np.real(np.vdot(final_state, hamiltonian @ final_state))


result = scipy.optimize.minimize(
    fun,
    x0=operator.to_parameters(
        alpha_alpha_indices=alpha_alpha_indices, alpha_beta_indices=alpha_beta_indices
    ),
)
print(f"Number of parameters: {len(result.x)}")
print(result)